* API KEY : https://developers.naver.com/apps/#/list
* API DOCUMENT : https://developers.naver.com/docs/serviceapi/search/shopping/shopping.md#%EC%87%BC%ED%95%91

In [ ]:
import os
import requests
from dotenv import load_dotenv
import pandas as pd

# 1. .env 파일에서 환경변수 읽어오기
load_dotenv()  # 현재 프로젝트 폴더(또는 지정한 경로)에 있는 .env 파일 로드

client_id = os.getenv('NAVER_CLIENT_ID')
client_secret = os.getenv('NAVER_CLIENT_SECRET')

# 2. 요청할 URL과 헤더 설정
base_url = "https://openapi.naver.com/v1/search/shop.json"
headers = {
    "X-Naver-Client-Id": client_id,
    "X-Naver-Client-Secret": client_secret
}

# 3. 검색어 설정
query = "운동화"

# 수집한 모든 아이템을 담을 리스트
all_items = []

# 1페이지부터 10페이지까지 반복
# (start 파라미터는 1, 101, 201, ... 901)
for page in range(1, 11):
    start_num = (page - 1) * 100 + 1  # 1, 101, 201, ...
    
    params = {
        "query": query,
        "display": 100,      # 한 번에 가져올 검색 결과 개수
        "start": start_num,  # 이번 루프에서의 검색 시작 위치
        "sort": "sim"        # 정확도순
    }
    
    # 4. API 호출
    response = requests.get(base_url, headers=headers, params=params)

    if response.status_code == 200:
        data = response.json()
        items = data.get('items', [])
        
        # 페이지별로 받아온 items를 누적
        all_items.extend(items)
        
        print(f"\n===== {page}페이지 (start={start_num}) 결과 =====")
        for i, item in enumerate(items, start=1):
            title = item.get('title')
            link = item.get('link')
            lprice = item.get('lprice')
            mall_name = item.get('mallName', '정보없음')
            
            # HTML 태그 제거 예시 (여기서는 간단히 replace 사용)
            # <b> 태그 등은 replace로 간단히 제거할 수도 있음
            clean_title = title.replace("<b>", "").replace("</b>", "")
            
            # print(f"[{i}] {clean_title} | 가격: {lprice} | {mall_name} | 링크: {link}")
    else:
        print(f"Error Code: {response.status_code}")
        print(response.text)
        break  # 에러 발생 시 반복 중단

# 최종적으로 가져온 전체 아이템 수
print(f"\n총 수집된 아이템 수: {len(all_items)}")

# DataFrame으로 변환(선택)
df = pd.DataFrame(all_items)

In [ ]:
df.shape